# Project with Solution (Section 11)

## Project Specs

Here is where we left off in the previous section code:

In [1]:
countries = {
    "australia": ("Australia", "AUS"),
    "canada": ("Canada", "CAN"),
    "china": ("China", "CHN"),
    "france": ("France", "FRA"),
    "germany": ("Germany", "DEU"),
    "india": ("India", "IND"),
    "mexico": ("Mexico", "MEX"),
    "norway": ("Norway", "NOR"),
    "pakistan": ("Pakistan", "PAK"),
    "san marino": ("San Marino", "SMR"),
    "sanmarino": ("San Marino", "SMR"),
    "spain": ("Spain", "ESP"),
    "sweden": ("Sweden", "SWE"),
    "united kingdom": ("United Kingdom", "GBR"),
    "uk": ("United Kingdom", "GBR"),
    "great britain": ("United Kingdom", "GBR"),
    "britain": ("United Kingdom", "GBR"),
    "us": ("United States of America", "USA"),
    "united states": ("United States of America", "USA"),
    "usa": ("United States of America", "USA"),
}
valid_country_names = sorted(countries.keys())

def lookup_country(name: str) -> tuple[str, str]:
    name = name.strip().casefold()
    
    try:
        return countries[name]
    except KeyError:
        raise ValueError(
            "Unknown country name. "
            f"Country name must be one of: {','.join(valid_country_names)}"
        )

country_code_lookup = {
    name: code
    for name, code in countries.values()
}

In [2]:
from datetime import date
from enum import Enum
from functools import cached_property
from typing import Annotated, TypeVar
from uuid import uuid4
from pydantic import (
    AfterValidator, 
    BaseModel, 
    ConfigDict, 
    computed_field,
    Field, 
    field_serializer, 
    field_validator,
    PlainSerializer,
    UUID4,
    ValidationInfo,
)
from pydantic.alias_generators import to_camel


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


T = TypeVar('T')
BoundedString = Annotated[str, Field(min_length=2, max_length=50)]
BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]
Country = Annotated[str, AfterValidator(lambda name: lookup_country(name)[0])]


def serialize_date(value: date) -> str:
        return value.strftime("%Y/%m/%d")


CustomDate = Annotated[
    date, 
    PlainSerializer(serialize_date, when_used="json-unless-none")
]

class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = Field(default=False, repr=False)
    manufactured_date: CustomDate = Field(
        validation_alias="completionDate", 
        ge=date(1980, 1, 1), 
        repr=False
    )
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD",
        repr=False,
    )
    top_features: BoundedList[BoundedString] | None = Field(default=None, repr=False)
    vin: BoundedString = Field(repr=False)
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
        repr=False,
    )
    registration_country: Country | None = Field(default=None, repr=False)

    @computed_field(repr=False)
    @cached_property
    def registration_country_code(self) -> str:
        return country_code_lookup[self.registration_country] 
    
    registration_date: CustomDate | None = Field(default=None, repr=False)
    license_plate: BoundedString | None = Field(default=None, repr=False)
        
    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

Here we are going to refactor our model and put the registration country information into a separate model, that we'll then include in our main model via composition.

But before we do that, we'll want to apply the same model configurations to all our models.

So, first create a new custom base model (call it `CamelBaseModel`), and define the same model config as our current `Automobile` model.

Then, use inheritance to remove the model configs from `Automobile`, and instead inherit them from our new `CamelBaseModel`.

Next, we are going to make a separate model (use the `CamelBaseModel` for this one too), that should move these fields from `Automobile` into it (including any validators). You may also want to clean up the representation settings for this model when you copy things over.

- implement the `registration_country` field in this new model - but rename the field to simply `name`.
- implement the computed field `registration_country_code` in this new model - but rename the field to simply `code3`.

Once you have that model defined (and removed the `registration_country` and `registration_country_code` fields from the `Automobile` model), create a single new replacement field, also named `registration_country`, but this time use the submodel as the type. Of course, this field should still have a default of `None`.

## Solution

First we define our custom base model:

In [3]:
class CamelBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

Then we can redefine our `Automobile` class this way:

In [4]:
class Automobile(CamelBaseModel):
    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = Field(default=False, repr=False)
    manufactured_date: CustomDate = Field(
        validation_alias="completionDate", 
        ge=date(1980, 1, 1), 
        repr=False
    )
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD",
        repr=False,
    )
    top_features: BoundedList[BoundedString] | None = Field(default=None, repr=False)
    vin: BoundedString = Field(repr=False)
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
        repr=False,
    )
    registration_country: Country | None = Field(default=None, repr=False)

    @computed_field(repr=False)
    @cached_property
    def registration_country_code(self) -> str:
        return country_code_lookup[self.registration_country] 
    
    registration_date: CustomDate | None = Field(default=None, repr=False)
    license_plate: BoundedString | None = Field(default=None, repr=False)
        
    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

Next, we define a new model for registration country:

In [5]:
class RegistrationCountry(CamelBaseModel):
    name: Country | None = Field(default=None)

    @computed_field
    @cached_property
    def code3(self) -> str:
        return country_code_lookup[self.name]

And now, using composition let's refactor the fields in `Automobile`:

In [6]:
class Automobile(CamelBaseModel):
    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = Field(default=False, repr=False)
    manufactured_date: CustomDate = Field(
        validation_alias="completionDate", 
        ge=date(1980, 1, 1), 
        repr=False
    )
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD",
        repr=False,
    )
    top_features: BoundedList[BoundedString] | None = Field(default=None, repr=False)
    vin: BoundedString = Field(repr=False)
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
        repr=False,
    )
    registration_country: RegistrationCountry | None = Field(default=None, repr=False)
    registration_date: CustomDate | None = Field(default=None, repr=False)
    license_plate: BoundedString | None = Field(default=None, repr=False)
        
    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

Finally, let's deserialize some data:

In [7]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": {"name": "us"},
    "registrationDate": "2023-06-01",
    "licensePlate": "AAA-BBB"
}

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': {
        'name': 'United States of America', 
        'code3': 'USA'
    },
    'registrationDate': date(2023, 6, 1),
    'licensePlate': 'AAA-BBB'
}

In [8]:
car = Automobile.model_validate(data)
car

Automobile(id_=UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'), manufacturer='BMW', series_name='M4 Competition xDrive', type_=<AutomobileType.convertible: 'Convertible'>)

In [9]:
car.registration_country

RegistrationCountry(name='United States of America', code3='USA')

And let's deserialize:

In [10]:
car.model_dump(by_alias=True)

{'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
 'manufacturer': 'BMW',
 'seriesName': 'M4 Competition xDrive',
 'type': <AutomobileType.convertible: 'Convertible'>,
 'isElectric': False,
 'manufacturedDate': datetime.date(2023, 1, 1),
 'baseMSRPUSD': 93300.0,
 'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
 'vin': '1234567890',
 'numberOfDoors': 2,
 'registrationCountry': {'name': 'United States of America', 'code3': 'USA'},
 'registrationDate': datetime.date(2023, 6, 1),
 'licensePlate': 'AAA-BBB'}

In [11]:
assert car.model_dump(by_alias=True) == expected_serialized_by_alias